In [1]:
!pip install langchain langchain_community pypdf openai

In [ ]:
%env API_KEY=替换为自己的
%env BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

import os

In [4]:
dt = '20240811'
version = 'v1_1'

output_dir = os.path.join('outputs', f'{version}_{dt}')
os.makedirs(output_dir, exist_ok=True)

In [5]:
loader = PyPDFLoader("data/2024全球经济金融展望报告.pdf")
documents = loader.load()

In [6]:
len(documents)

53

In [7]:
len(documents[0].page_content)

643

In [8]:
import re

pattern = r"^全球经济金融展望报告\n中国银行研究院 \d+ 2024年"
# re.sub(pattern, '', splitted_docs_large[3].page_content)

In [9]:
merged_docs = [Document(page_content='\n'.join(re.sub(pattern, '', doc.page_content) for doc in documents))]

In [10]:
from uuid import uuid4
import os
import pickle

from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, filepath, chunk_size=400, chunk_overlap=40, seperators=['\n\n\n', '\n\n'], force_split=False):
    if os.path.exists(filepath) and not force_split:
        print('found cache, restoring...')
        return pickle.load(open(filepath, 'rb'))

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=seperators
    )
    split_docs = splitter.split_documents(documents)
    for chunk in split_docs:
        chunk.metadata['uuid'] = str(uuid4())

    pickle.dump(split_docs, open(filepath, 'wb'))

    return split_docs

In [11]:
splitted_docs = split_docs(documents, os.path.join(output_dir, 'split_docs.pkl'), chunk_size=500, chunk_overlap=50)
splitted_docs_large = split_docs(merged_docs, os.path.join(output_dir, 'split_docs_large.pkl'), chunk_size=1500, chunk_overlap=100)

In [12]:
len(splitted_docs), len(splitted_docs_large)

(52, 28)

In [13]:
uuid2doc = {doc.metadata['uuid']: doc.page_content for doc in splitted_docs}
uuid2large_doc = {doc.metadata['uuid']: doc.page_content for doc in splitted_docs_large}

# 抽取QA

In [14]:
qa_gen_prompt_tmpl = """
我会给你一段文本（<document></document>之间的部分），你需要阅读这段文本，分别针对这段文本生成8个问题、用户回答这个问题的上下文，和基于上下文对问题的回答。

对问题、上下文、答案的要求：

问题要与这段文本相关，不要询问类似“这个问题的答案在哪一章”这样的问题
上下文：上下文必须与原始文本的内容保持一致，不要进行缩写、扩写、改写、摘要、替换词语等
答案：回答请保持完整且简洁，无须重复问题。答案要能够独立回答问题，而不是引用现有的章节、页码等

返回结果以JSON形式组织，格式为[{"question": "...", "context": ..., "answer": "..."}, ...]。
如果当前文本主要是目录，或者是一些人名、地址、电子邮箱等没有办法生成有意义的问题时，可以返回[]。

下方是文本：
<document>
{{document}}
</document>

请生成结果：
"""

qa_gen_prompt_tmpl_large_context = """
我会给你一段文本（<document></document>之间的部分），你需要阅读这段文本，分别针对这段文本生成2个问题，和基于这段文本对问题的回答，回答请保持完整，无须重复问题。
尽可能创建一些需要综合*大段*文本才能回答的问题，但不要问类似“这一段主要讲了什么内容”这样的问题，答案要能够独立回答问题，而不是引用现有的章节、页码等；不要问具体过于细节的问题，例如“海湾国家的2024年预期经济增长率是多少”，而是尽可能问类似“2024年全球经济的几大趋势是什么”、“受局部中东地区紧张局势影响，可能对全球原物料有哪些影响”。
返回结果以JSON形式组织，格式为[{"question": "...", "answer": "..."}, ...]。
如果当前文本主要是目录，或者是一些人名、地址、电子邮箱等没有办法生成有意义的问题时，可以返回[]。

下方是文本：
<document>
{{document}}
</document>

请生成结果：
"""

In [15]:
# prompt_tmpl2 = """
# 请帮我基于这份文档，构造10条数据，包括问题和答案和关键词，尽量生成一些需要全局信息的问题，回答尽量详细一些。
# 请注意，答案必须来自于此文档，结果请以JSON数组形式组织，格式为[{"question": "...", "answer": "...", "keywords": ["..."]}]
# """

# prompt_tmpl3 = """
# 请帮我基于这份文档，构造20条数据，包括问题和答案和关键词，尽量生成一些询问流程、有哪些步骤、有哪些组成部分的问题，回答尽量详细一些。
# 请注意，答案必须来自于此文档，结果请以JSON数组形式组织，格式为[{"question": "...", "answer": "...", "keywords": ["..."]}]
# """

In [16]:
from openai import OpenAI
import time
import random

client = OpenAI(
    api_key=os.environ['API_KEY'],
    base_url=os.environ['BASE_URL']
)

def build_qa_prompt(prompt_tmpl, text):
    prompt = prompt_tmpl.replace('{{document}}', text).strip()
    return prompt

def chat(prompt, max_retry=3, debug=False, model_name='qwen-long', temperature=0.85, top_p=0.95):
    def do_chat(prompt):
        completion = client.chat.completions.create(
            # model="Qwen/Qwen2-72B-Instruct",
            model=model_name,
            messages=[    
                {"role": "system", "content": "你是一个有用的人工智能助手"},    
                {"role": "user", "content": prompt}
            ],
            top_p=top_p,
            temperature=temperature
        )
        return completion.choices[0].message.content

    while max_retry > 0:
        try:
            return do_chat(prompt)
        except Exception as e:
            max_retry -= 1
            sleep_seconds = random.randint(1, 4)
            if debug:
                print(f"{str(e)}, remain retry: {max_retry}, sleeping {sleep_seconds}s {prompt}")
            time.sleep(sleep_seconds)
    return None

In [17]:
text = splitted_docs[40].page_content

In [18]:
print(text)

全球经济金融展望报告
中国银行研究院 40 2024年
注重绿色经济发展，为中海经贸合作创造了良好机遇。
（一）近年来海湾六国经济表现受能源价格影响较大
受国际能源价格走势影响，近年来海湾六国经济增长波动较大。2022年，
在全球油气价格飙升推动下，海湾六国经济增长强劲，实际GDP增速达7.3%。
其中，沙特占据海湾六国经济体量的半壁江山。2022年，沙特GDP规模达1.1
万亿美元，阿联酋和卡塔尔GDP规模分别为5075亿和2373亿美元，位列第二
和第三位（图26）。油气部门在海湾六国经济总量中的占比超过40%，在沙特
的占比更是接近七成。
图26：2022年海湾六国经济体量（亿美元）
资料来源：世界银行，中国银行研究院
2023年以来，海湾六国经济增长明显放缓。主要经济体货币政策加速收紧、
全球经济活动减少使全球能源需求下降，海湾六国经济增长面临严重挑战。IMF
在最新展望报告中将海湾六国2023年经济增速下调0.8个百分点至1.7%。其中，
沙特经济增速预计仅为0.8%，低于中东石油出口国的平均水平。相比之下，阿
联酋韧性较强，预计2023年和2024年经济增速分别为3.4%和4.0%，为海湾
六国中最高（图27）。鉴于海湾六国经济结构相对单一，非石油部门虽然为海


In [19]:
print(chat(build_qa_prompt(qa_gen_prompt_tmpl, text), debug=True))

```json
[
  {
    "question": "近年来海湾六国的经济表现受到什么因素的影响较大？",
    "context": "受国际能源价格走势影响，近年来海湾六国经济增长波动较大。",
    "answer": "近年来海湾六国的经济表现受国际能源价格的影响较大。"
  },
  {
    "question": "2022年海湾六国的实际GDP增速是多少？",
    "context": "2022年，在全球油气价格飙升推动下，海湾六国经济增长强劲，实际GDP增速达7.3%。",
    "answer": "2022年海湾六国的实际GDP增速达到7.3%。"
  },
  {
    "question": "2022年沙特的GDP规模是多少？",
    "context": "2022年，沙特GDP规模达1.1万亿美元。",
    "answer": "2022年沙特的GDP规模为1.1万亿美元。"
  },
  {
    "question": "2022年阿联酋和卡塔尔的GDP规模分别是多少？",
    "context": "阿联酋和卡塔尔GDP规模分别为5075亿和2373亿美元。",
    "answer": "2022年阿联酋的GDP规模为5075亿美元，卡塔尔为2373亿美元。"
  },
  {
    "question": "2023年以来海湾六国经济增长发生了什么变化？",
    "context": "2023年以来，海湾六国经济增长明显放缓。",
    "answer": "2023年以来海湾六国的经济增长明显放缓。"
  },
  {
    "question": "IMF将2023年海湾六国经济增速下调至多少？",
    "context": "IMF在最新展望报告中将海湾六国2023年经济增速下调0.8个百分点至1.7%。",
    "answer": "IMF将2023年海湾六国经济增速下调至1.7%。"
  },
  {
    "question": "2023年沙特的经济增速预计会是多少？",
    "context": "沙特经济增速预计仅为0.8%，低于中东石油出口国的平均水平。",
    "answer": "2023年沙特的经济增速预计仅为0.8%。"
  }

In [20]:
print(splitted_docs_large[8].page_content)

存在专向性1或属于禁止性补贴，且对欧盟内部产业构成实质性损害的情况下，欧盟即可认定补贴属于
可诉性补贴。在此情况下，欧盟一般可能采取两类反补贴措施：一是加征反补贴税，针对不同类型的
企业，适用的加征税率也有所差异，其中被认定为不合作企业将会被征收最高税率；二是出口商接受
价格承诺，以不低于特定价格出口。反补贴措施一般实施五年。当欧盟认定补贴未对其产业带来损害、
补贴微量或申请人撤诉的情况下，可以无措施结案。
欧盟从2010年首次发起对华反补贴调查开始，共计对华发起过18起反补贴调查，涉及多个制造
业领域，其中接近三分之二的调查最终采取了加征关税的措施。因此，从历史经验看，反补贴调查一
经发起，欧盟较大概率最终将采取加征关税措施。
3.日本经济总体保持稳定，货币政策有望迎来拐点
2023年日本经济整体呈现先升后降走势。上半年，在疫后重新开放市场、
供应链修复等因素影响下，日本经济实现较快增长，一、二季度GDP环比增长
折年率分别达到3.7%和4.5%。但三季度日本经济增长形势急转直下，GDP环
比折年率降至-2.1%，三个季度以来首次出现负增长。从全年情况看，通胀压力
和疲软的日元抑制了私人需求，占日本GDP近六成的个人消费连续两个季度出
现负增长，严重拖累日本经济。前期对日本经济增长贡献较大的外需在三季度
也逐渐失去活力，特别是前期增长旺盛的访日外国游客消费在三季度出现5%
的环比下滑，同期日本进口出现三个季度的首次正增长，净出口对整体经济的
贡献由正转负。民间设备投资在二季度下降1%后三季度再下降0.6%，企业在
价格压力不减的情况下继续削减投资。政府消费经历先降后升走势，日本政府
财政政策在三季度开始发力，但仍不足以支撑日本经济整体表现。按照平均汇
率折算，前三季度日本经济总量被德国赶超，降至全球第四位。
1如果补贴授予当局补贴发放明确限定于特定企业，这种补贴即具有专向性。

图11：日本GDP环比增长折年率变化情况（%）
资料来源：Wind，中国银行研究院
日本通胀压力较大。2022年以来，日本通胀持续上涨，并于2023年初达
到41年来高位。此后受益于能源进口价格下滑以及政府对抑制电费和燃气费的
补贴政策，通胀水平有所回落，但最新数据仍在3%左右的水平震荡，显著高于
过去二十多年0.15%的平均水平。日本对海外能源、食品的依赖程度较为突出，
特别是能源进口依

In [21]:
print(chat(build_qa_prompt(qa_gen_prompt_tmpl_large_context, splitted_docs_large[8].page_content)))

[
{"question": "欧盟在什么条件下会认定补贴为可诉性补贴并采取反补贴措施?", "answer": "当补贴存在专向性或属于禁止性补贴,且对欧盟内部产业构成实质性损害的情况下,欧盟即可认定补贴属于可诉性补贴。在这种情况下,欧盟一般可能采取两类反补贴措施:一是加征反补贴税,针对不同类型的企业的加征税率也会有所不同,其中不合作企业将会被征收最高税率;二是出口商接受价格承诺,以不低于特定价格出口。"},
{"question": "2023年日本经济的表现如何,尤其是其经济增长和个人消费方面的特点?", "answer": "2023年日本经济整体呈现先升后降的走势,一、二季度GDP环比增长折年率分别达到3.7%和4.5%,但三季度降至-2.1%。全年来看,通胀压力和疲软的日元抑制了私人需求,个人消费连续两个季度出现负增长,严重影响日本经济。此外,外需在三季度逐渐失去活力,特别是访日外国游客消费在三季度出现5%的环比下滑,同时,政府消费经历先降后升走势,但仍不足以支撑日本经济整体表现。"}
]


In [22]:
import threading
import concurrent.futures
from tqdm.auto import tqdm
import json

def gen_qa(splitted_docs, prompt_tmpl, qa_ckpt_filename):
    qa_ckpt = {}
    if os.path.exists(qa_ckpt_filename):
        qa_ckpt = open(qa_ckpt_filename).readlines()
        qa_ckpt = [json.loads(line.strip()) for line in qa_ckpt if line.strip() != '']
        qa_ckpt = {item['uuid']: item for item in qa_ckpt}
        print(f'found checkpoint, item count: {len(qa_ckpt)}')
    
    file_lock = threading.Lock()
    max_workers = 4
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {doc.metadata['uuid']: executor.submit(chat, build_qa_prompt(prompt_tmpl, doc.page_content), 3, True) for doc in splitted_docs if len(doc.page_content.replace('\n', '')) >= 150 and doc.metadata['uuid'] not in qa_ckpt}
        for uuid in tqdm(futures):
            future = futures[uuid]
            result = future.result()
            if result is None:
                continue
            
            item = {'uuid': uuid, 'raw_resp': result}
            qa_ckpt[uuid] = item
    
            # global file_lock
            file_lock.acquire()
            
            try:
                with open(qa_ckpt_filename, 'a') as f:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')
            except Exception as e:
                print(e)
            finally:
                file_lock.release()
    return qa_ckpt

In [23]:
detailed_qa_dict = gen_qa(splitted_docs, qa_gen_prompt_tmpl, os.path.join(output_dir, f"qa_ckpt_detailed.jsonl"))

  0%|          | 0/49 [00:00<?, ?it/s]

In [24]:
large_context_qa_dict = gen_qa(splitted_docs_large, qa_gen_prompt_tmpl_large_context, os.path.join(output_dir, f"qa_ckpt_large_context.jsonl"))

  0%|          | 0/28 [00:00<?, ?it/s]

# 后置处理

In [25]:
import re

def convert2json(text):
    # pattern = r'```\[(.*)\]```'
    # matched = match.group(1)
    pattern = r'\[.*\]'
    # text = qa_ckpt['8e2d7b02-4207-47b1-a8a5-14409b1b86f6']['raw_resp']

    text = text.replace('>>>', '')
    try:
        return json.loads(text)
    except:
        match = re.search(pattern, text, re.DOTALL)
        try:
            matched = match.group(0)
            return json.loads(matched)
        except Exception as e:
            print(f"{match}, {str(e)}")

    return []

In [26]:
from tqdm.auto import tqdm
import pandas as pd
import pickle

def build_qa_df(qa_ckpt, uuid2doc_map):
    data = []
    
    for key, value in tqdm(qa_ckpt.items()):
        text = value['raw_resp']
        qa_list = convert2json(text)
    
        for item in qa_list:
            question = item.get('question', '').strip()
            answer = item.get('answer', '').strip()
            context = item.get('context', '').strip()
            
            if question == '' or answer == '':
                print(qa_list)
                continue
            data.append({
                'uuid': key,
                'question': question,
                'answer': answer,
                'context': context,
                'doc': uuid2doc_map[key]
            })
    qa_df = pd.DataFrame(data)
    return qa_df

In [27]:
qa_df = build_qa_df(detailed_qa_dict, uuid2doc)
large_context_qa_df = build_qa_df(large_context_qa_dict, uuid2large_doc)

  0%|          | 0/49 [00:00<?, ?it/s]

[{'question': '2024年对欧洲经济增长的预期被下调了几次?', 'context': '这是两个多月的时间里第二次下调对欧洲经济增长的预期。', 'answer': '两次'}, {'question': '英国预算责任办公室对英国2024年的经济增长预测是多少百分比?', 'context': '1.4%'}, {'question': '10月份欧元区经济景气指数是多少?', 'context': '10月欧元区经济景气指数为93.3点。', 'answer': '93.3点'}, {'question': '未来欧元区经济增长面临的潜在挑战有哪些?', 'context': '未来应警惕取暖季天然气价格再度飙升风险以及南欧部分经济体预算紧张局势对欧元区经济增长的潜在挑战。', 'answer': '取暖季天然气价格再度飙升风险以及南欧部分经济体预算紧张局势'}, {'question': '2024年欧元区和英国的经济增速预计分别是多少?', 'context': '预计2023年欧元区和英国经济增速分别为0.6%和0.3%左右，2024年经济增速将分别达0.9%和0.5%左右。', 'answer': '欧元区为0.9%，英国为0.5%'}, {'question': '2023年10月欧盟委员会对中国的哪个行业发起了首次反补贴调查?', 'context': '2023年10月，欧盟委员会决定依职权对中国电动汽车发起反补贴调查。', 'answer': '电动汽车'}, {'question': '欧盟通常会如何认定构成补贴的情况?', 'context': '根据WTO《补贴与反补贴措施协定》规定以及欧盟过去对华反补贴调查的实践，被认定为补贴的情况通常包括政府拨款、税收优惠、优惠融资以及低价获得土地、原材料或服务等。', 'answer': '政府拨款、税收优惠、优惠融资以及低价获得土地、原材料或服务等'}, {'question': '2010年至2023年间，欧盟对中国哪一种产品发起了反补贴调查?', 'context': '涉案产品 发起时间...电动汽车 2023/10', 'answer': '铜版纸、数据卡、彩涂板、自行车、光伏、太阳能玻璃、玻璃纤维长丝、聚酯短纤维、冷轧不锈钢、热轧卷板、卡客车轮胎、电动自行车、玻璃纤维织物、

  0%|          | 0/28 [00:00<?, ?it/s]

None, 'NoneType' object has no attribute 'group'
<re.Match object; span=(0, 412), match='[\n  {\n    "question": "2023年发展中国家在货币合作方面取得了哪些实质>, Expecting property name enclosed in double quotes: line 5 column 3 (char 214)
[{'question': '根据文本所提供的信息，美国房地产市场的哪些因素导致了房贷申请的下降?', 'answer': '美国房价持续上升叠加房贷利率上升，这两大因素影响了购房需求，导致房贷申请下降。此外，大部分负债购房发生在低端市场，而中端市场由于住房可负担性的严重受挫已基本冻结，这些都进一步抑制了房贷申请的数量。'}, {'question': '文本中提到了几个方面来反映美国当前的住房负担状况：一是美国家庭全部债务支出与可支配收入的比例在疫情期间上升，但整体低于历史水平；二是固定利率类房贷占主导地位，使得债务人的月供负担较低；三是计划购房者中有71%表示当房贷利率超过5.5%时将不再考虑购房；四是待售房屋供应紧张，导致房价持续上涨，进而降低了住房的负担能力。'}]


In [28]:
len(qa_df)

383

In [29]:
qa_df.drop_duplicates('question', inplace=True)

In [30]:
len(qa_df)

383

In [31]:
len(large_context_qa_df)

51

In [32]:
large_context_qa_df.drop_duplicates('question', inplace=True)

In [33]:
len(large_context_qa_df)

51

In [34]:
qa_df.sample(5)

,uuid,question,answer,context,doc
112,ae74431c-0b26-4f76-803c-21159e5ec4c1,2023年10月欧元区哪些领域的价格涨幅仍然明显?,食品、酒精和烟草价格、服务业和非能源工业产品价格涨幅仍然明显。,10月欧元区食品、酒精和烟草价格、服务业和非能源工业产品价格涨幅仍然明显，但能源价格同比大幅...,全球经济金融展望报告\n中国银行研究院 14 2024年\n降，较欧元区而言英国通胀更加顽固...
308,96aec7da-2ee3-4e25-9955-15ecbb8d26ad,IMF将2023年海湾六国经济增速下调至多少？,IMF将2023年海湾六国经济增速下调至1.7%。,IMF在最新展望报告中将海湾六国2023年经济增速下调0.8个百分点至1.7%。,全球经济金融展望报告\n中国银行研究院 40 2024年\n注重绿色经济发展，为中海经贸合作...
75,e9ef727f-6276-4854-b77c-546106a48689,2023年前三个季度美国服务消费的增长主要归因于什么？,2023年前三个季度美国服务消费的增长主要归因于消费者在服务领域的需求释放及稳健的就业市场。,后疫情时代消费者在服务领域的压抑需求\n得到释放。服务支出成为消费者支出领域中最稳定和增幅最...,全球经济金融展望报告\n中国银行研究院 9 2024年\n综合上述因素，“复苏+分化”将成为...
224,a476e490-cb06-46c8-b67f-160054866fe8,2023年11月15日美元指数是多少?,2023年11月15日美元指数为104.4。,2023年11月15日，美元指数为104.4，较年初上涨0.6%。,全球经济金融展望报告\n中国银行研究院 29 2024年\n（三）汇率市场：美元指数高位震荡...
211,d1df0c0e-2dbd-4c19-896e-4d3ea941cc3c,2023年欧元区组合投资净流入规模是多少?,2023年欧元区组合投资净流入规模为531亿欧元,2023年前三季度，欧元区非储备性质金融账户净流出规模达2143亿欧元，其中直接投资净流出4...,全球经济金融展望报告\n中国银行研究院 27 2024年\n比增速分别为-1.04%、-2....


In [35]:
qa_df['qa_type'] = 'detailed'

In [36]:
large_context_qa_df.sample(5)

,uuid,question,answer,context,doc
16,d09602e4-4b94-4095-84bf-5a29fe305720,2024年哪些因素有望支持日本经济的增长?,"2024年,财政扩张、工资增长和商业信心回暖有望支撑日本内需转向正增长。随着库存调整结束,预...",,图12：日本通胀同比增速变化（%）\n资料来源：Wind，中国银行研究院\n展望2024年，...
0,31c7b58a-cd60-49b9-9860-0d4d771a42b2,2023年全球经济和金融的总体表现如何？,2023年全球经济增长动力持续回落，各国复苏分化，发达经济体增速明显放缓，新兴经济体整体表现...,,研究院\n全球经济金融展望报告\n要点2024年年报（总第57期） 报告日期：2023年12...
2,67ed758b-aedc-4969-a09a-68cefa312c02,2023年全球投资率相比于2022年有何变化？预测下降了多少个百分点？,IMF预测2023年全球投资率（投资占GDP的比重）将下降1.0个百分点至26.4%。,,图3：部分经济体工业生产指数变化趋势（2010年=100）\n资料来源：荷兰经济分析局，中国...
49,6cf0d4aa-5a9f-4982-93b5-a67b84e3a18e,商业房地产市场面临哪些挑战导致其风险上升?,"商业房地产市场面临的挑战包括:贷款新发放量大幅下降,CMBS新发行出现停滞;商业房地产价格指...",,该类贷款新发放量将同比下降46%。同时，商业房地产抵押证券CMBS新发行\n也出现停滞。商业...
40,b6cc8ba4-3a69-4cda-a711-0e13fddf9805,海湾六国在新能源转型方面采取了哪些措施?,海湾六国已采取多项措施推动新能源转型。五国已设定净零排放目标，其中阿联酋的目标是在2050年...,,供了空间，进一步坚定了海湾六国经济转型的决心。\n近年来，海湾六国努力推进新能源转型，增强国...


In [37]:
large_context_qa_df['qa_type'] = 'large_context'

In [38]:
qa_df = pd.concat([qa_df, large_context_qa_df])

In [39]:
qa_df.sample(5)

,uuid,question,answer,context,doc,qa_type
351,5c2a9e5a-e97e-48d2-a630-d569aa542e44,银行对于居住房地产信贷的标准如何?,银行对所有的居住房地产信贷，包括次级房贷的标准较为严格，很少有银行放松标准。,高级信贷官员的考察发现，银行对所有的居住房地产信贷，包括次级房贷的标准均较为严格，很少有银行...,全球经济金融展望报告\n中国银行研究院 46 2024年\n高级信贷官员的考察发现，银行对所...,detailed
374,b0ef3b31-0d8e-4a49-b7fb-0227a60c6b1f,与居住房地产相比，商业房地产及其信贷对哪些因素更为敏感?,与居住房地产相比，商业房地产及其信贷对价格下跌、流动性收紧更为敏感,与居住房地产相比，商业房地产及其信贷对价格下跌、流动性收紧更为敏感，其下行周期延续时间可能更长。,全球经济金融展望报告\n中国银行研究院 48 2024年\n该类贷款新发放量将同比下降46%...,detailed
100,ad4de2d7-ffd7-499a-81a4-6288f91c5af3,2023年10月欧元区综合PMI数值及意义是什么?,2023年10月欧元区综合PMI下跌至46.5%，是自2020年11月以来的最低点，这表明商...,10月欧元区综合PMI进一步下跌至46.5%，创2020年11月以来新低，商业预期和新订单指...,全球经济金融展望报告\n中国银行研究院 12 2024年\n资支出和商业活动复苏。综合来看，...,detailed
180,edb36687-c8dd-47cd-9460-fb02c158bef5,哪些新兴经济体在2023年第四季度进一步提升了利率水平？,2023年第四季度，土耳其、俄罗斯和印尼的央行分别加息500、200和25个基点，提升了本国...,与发达经济体相反，部分新兴经济体在四季度进一步提升利率水平，其中，土耳其央行10月大幅加息5...,全球经济金融展望报告\n中国银行研究院 23 2024年\n图14：影响国际金融动态的六大因...,detailed
177,edb36687-c8dd-47cd-9460-fb02c158bef5,2023年欧洲央行的主要再融资利率有何变化？,2023年，欧洲央行前三个季度累计加息6次共175个基点，并于10月26日宣布维持主要再融资...,欧洲央行在前三个季度连续加息6次，累计175个基点，10月26日宣布维持主要再融资利率、存款...,全球经济金融展望报告\n中国银行研究院 23 2024年\n图14：影响国际金融动态的六大因...,detailed


In [40]:
len(qa_df)

434

# QA质量检查

In [41]:
qa_check_prompt_tmpl = """
你是一个金融领域的专家，现在有人根据一份经济发展报告，构造了一些问题，并对问题进行了回答。
你的任务是对这些问题（<question></question>之间的部分）和回答（<answer></answer>）进行打分。

结果请以JSON形式组织，格式如下（<result></result>之间的部分）：
<result>
{"score": ..., "reason": ...}
</result>
其中score是对问题-回答的打分，分值是一个int类型的值，取值范围为1-5。reason是打分的理由。

好的问题，应该是询问事实、观点等，不好的问题，通常要求做一些文本摘要等初级文字处理工作，类似于“这一段描述了什么”，“文本描述了什么”；或者询问的内容是图相关的，例如“图4展示了什么数据？”。
好的答案，应该能够回应问题，而不是回答无关的内容，不好的回答，会给出在原文中的引用，例如“第3章”等。

问题：
<question>
{{question}}
</question>

参考答案：
<answer>
{{answer}}
</answer>

请进返回JSON格式的数据即可，不要添加其他任何描述性信息。
"""

In [42]:
def build_qa_scoring_prompt(row):
    context = row['context']
    question = row['question']
    answer = row['answer']
    prompt = qa_check_prompt_tmpl.replace('{{question}}', question).replace('{{answer}}', answer)

    return prompt

在大规模打分之前，先调试一下，检查Prompt是否正常

In [43]:
qa_df.iloc[41].to_dict()

{'uuid': '0c244063-29cd-45bd-b3d3-9b768e4d7c15',
 'question': '2023年三季度美国私人投资的表现如何?',
 'answer': '2023年三季度美国私人投资有所增长，增速提升至8.4%（经季调后环比折年率）。不过，制造业和设备投资增速则出现了放缓。',
 'context': '美国私人投资在2023年一季度触底后逐渐反弹，三季度存货及住宅投资恢复增长，带动私人投资增速提升至8.4%（经季调后环比折年率），但制造业和设备投资均放缓，环比增长折年率分别降低0.1%和3.8%。',
 'doc': '全球经济金融展望报告\n中国银行研究院 5 2024年\n图4：部分欧洲国家零售销售指数\n注：除英国是以2019年为基年外，其他经济体均为2015年为基年。\n资料来源：Wind，中国银行研究院\n发达经济体投资受加息政策影响较大，国内投资和跨境投资均持续承压。\n美国私人投资在2023年一季度触底后逐渐反弹，三季度存货及住宅投资恢复增\n长，带动私人投资增速提升至8.4%（经季调后环比折年率），但制造业和设备\n投资均放缓，环比增长折年率分别降低0.1%和3.8%。欧盟投资增速放缓，房\n地产投资减少。2023年二季度，欧元区固定资本形成总额环比增长0.1%，比一\n季度增速下降0.3个百分点，房地产对GDP环比增长拉动率转为负值。在紧缩\n货币政策影响下，发达经济体企业部门宏观杠杆率下降，企业加杠杆或负债投\n资意愿不足。同2022年底相比，2023年二季度，美国、英国、法国、意大利\n和德国非金融企业部门负债率分别下降了2.4个、3.4个、4.0个、3.0个和1.3\n个百分点（图5）。IMF预测2023年全球投资率（投资占GDP的比重）将下\n降1.0个百分点至26.4%（图6），其中，欧盟将下降1.1个百分点，比发达经\n济体平均降幅高0.2个百分点。从跨境投资角度看，受地缘政治局势紧张、金\n融领域动荡加剧、高利率和投资审查趋严等影响，并购交易仍然疲软，而在全\n球产业链重塑背景下，东南亚等区域绿地投资恢复增长。联合国贸发会议预计',
 'qa_type': 'detailed'}

In [44]:
print(build_qa_scoring_prompt(qa_df.iloc[41]))


你是一个金融领域的专家，现在有人根据一份经济发展报告，构造了一些问题，并对问题进行了回答。
你的任务是对这些问题（<question></question>之间的部分）和回答（<answer></answer>）进行打分。

结果请以JSON形式组织，格式如下（<result></result>之间的部分）：
<result>
{"score": ..., "reason": ...}
</result>
其中score是对问题-回答的打分，分值是一个int类型的值，取值范围为1-5。reason是打分的理由。

好的问题，应该是询问事实、观点等，不好的问题，通常要求做一些文本摘要等初级文字处理工作，类似于“这一段描述了什么”，“文本描述了什么”；或者询问的内容是图相关的，例如“图4展示了什么数据？”。
好的答案，应该能够回应问题，而不是回答无关的内容，不好的回答，会给出在原文中的引用，例如“第3章”等。

问题：
<question>
2023年三季度美国私人投资的表现如何?
</question>

参考答案：
<answer>
2023年三季度美国私人投资有所增长，增速提升至8.4%（经季调后环比折年率）。不过，制造业和设备投资增速则出现了放缓。
</answer>

请进返回JSON格式的数据即可，不要添加其他任何描述性信息。



In [45]:
print(chat(build_qa_scoring_prompt(qa_df.iloc[41]), debug=True, model_name='qwen-long', temperature=0))

{"score": 5, "reason": "问题针对具体时间与指标进行询问，回答直接相关且提供了具体信息。"}


In [46]:
import threading
import concurrent.futures
from tqdm.auto import tqdm
import json

qa_scoring_ckpt = {}
qa_scoring_ckpt_filename = os.path.join(output_dir, f"qa_scoring_ckpt.jsonl")
if os.path.exists(qa_scoring_ckpt_filename):
    qa_scoring_ckpt = open(qa_scoring_ckpt_filename).readlines()
    qa_scoring_ckpt = [json.loads(line.strip()) for line in qa_scoring_ckpt if line.strip() != '']
    qa_scoring_ckpt = {item['question']: item for item in qa_scoring_ckpt}
    print(f'found checkpoint, item count: {len(qa_scoring_ckpt)}')

file_lock = threading.Lock()
max_workers = 3
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {row['question']: executor.submit(chat, build_qa_scoring_prompt(row), 3, True, 'qwen-long', 0) for _, row in qa_df.iterrows() if row['question'] not in qa_scoring_ckpt}
    for question in tqdm(futures):
        future = futures[question]
        result = future.result()
        if result is None:
            continue
        
        item = {'question': question, 'raw_resp': result}
        qa_scoring_ckpt[question] = item

        global file_lock
        file_lock.acquire()
        
        try:
            with open(qa_scoring_ckpt_filename, 'a') as f:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        except Exception as e:
            print(e)
        finally:
            file_lock.release()

  0%|          | 0/434 [00:00<?, ?it/s]

In [47]:
def extract_json(text):
    """
    从raw_resp中提取出打分结果的JSON
    :param raw_resp: LLM的返回结果，数据样例：
    {\n"score": 5, \n"reason": "问题提出了一个具体且明确的观点询问，即针对美国房地产市场的风险评估，特别是对中小型银行的影响。参考答案直接回答了问题，并提供了详细的解释和原因，没有引用原始文本的位置，而是直接给出了分析性的内容。"\n}
    Based on the given criteria:\n\n- The question asks for a factual assessment of global trade performance in 2023, which is clear and direct.\n- The provided answer directly addresses the question without referring back to a source or chapter, offering an evaluation of global trade conditions in 2023.\n\nBoth the question and the answer meet the criteria for being high-quality and appropriately structured. Therefore, I would give them a high score.\n\n```json\n{"score": 5, "reason": "The question is clear and direct, seeking a factual evaluation. The answer provides a direct response without referring to specific sources or sections."}\n```
    """
    # pattern = r'\n```json\n(.*?)\n```'
    pattern = r'\{.*?\}'
    
    ret = {}
    try:
        ret = json.loads(text)
    except:
        match = re.search(pattern, text, re.DOTALL)
        try:
            matched = match.group(0)
            ret = json.loads(matched)
        except Exception as e:
            print(f"{match}, {str(e)}")
            
    return ret

In [48]:
qa_scoring_dict = {}

for key, value in qa_scoring_ckpt.items():
    try:
        qa_scoring_dict[key] = extract_json(value['raw_resp'])
        if 'score' not in qa_scoring_dict[key]:
            qa_scoring_dict[key] = {'score': -1, 'reason': f"parse failed, raw resp: {value['raw_resp']}"}
            raise ValueError(f'no score in result, question: {key}')
    except Exception as e:
        print(f"{key}, error: {e}")

In [49]:
qa_df['score'] = qa_df['question'].apply(lambda q: qa_scoring_dict.get(q, {}).get('score', -1))
qa_df['score_reason'] = qa_df['question'].apply(lambda q: qa_scoring_dict.get(q, {}).get('reason', -1))

In [50]:
qa_df.sample(5)

,uuid,question,answer,context,doc,qa_type,score,score_reason
231,2c419b1e-8adb-4be2-a70a-67ae15746376,2024年美元汇率的预期变化趋势如何？,美元在2024年前期保持强劲势头，之后随着美联储货币政策转向而逐步走弱，在下半年将呈现贬值态势。,在美国经济展现出较强韧性背景下，美元在2024年前期保持强劲势头，之后将随着美联储货币政策转...,全球经济金融展望报告\n中国银行研究院 30 2024年\n图18：主要货币兑美元汇率较20...,detailed,5,问题针对具体的时间点和主题提出了明确的询问，回答也直接针对问题给出了相应的预测，没有出现引用...
132,6eb03ba2-7a40-447f-a6a0-92ad97737976,2023年第三季度日本经济的表现如何?,2023年第三季度，日本经济出现负增长，GDP环比折年率为-2.1%。,但三季度日本经济增长形势急转直下，GDP环比折年率降至-2.1%，三个季度以来首次出现负增长。,全球经济金融展望报告\n中国银行研究院 16 2024年\n存在专向性1或属于禁止性补贴，且...,detailed,5,问题针对具体时间点的经济表现，回答直接且准确地给出了该时间段内的经济情况，符合良好问答的标准。
66,6fe7702d-fe61-465b-94b6-3ceb11208536,2024年欧洲和美国的经济表现预计如何？,欧元区和英国面临的通胀压力较大，经济表现将弱于其他主要发达经济体；美国经济处于下行阶段，但经...,相较于新兴市场和发展中经济体，发达经济体面临更大的经济增长压力。,全球经济金融展望报告\n中国银行研究院 8 2024年\n率政策。\n二是各国经济走势将进一...,detailed,5,问题针对未来经济表现提出了明确的询问，回答也直接针对问题给出了具体的预测，没有提及无关内容或...
194,684c8686-1a2f-4c4f-a453-e05ef233be43,2023年影响金融稳定性的因素有哪些?,2023年影响金融稳定性的主要因素包括：通胀维持高位、美联储持续大幅收紧货币政策、地缘冲突等。,2023年影响金融稳定性的主要因素包括：通胀维持高位、美联储持续大幅收紧货币政策、地缘冲\n突等。,全球经济金融展望报告\n中国银行研究院 25 2024年\n图15：美联储资产规模变化（亿美...,detailed,5,问题针对具体的时间点询问关于金融稳定性的因素，这是一个有效且明确的问题。回答直接针对问题提供...
143,74dd56a4-2bfe-4184-b7d4-75aebb93b5a7,哪些因素可能支撑2024年日本内需转向正增长?,财政扩张、工资增长和商业信心回暖有望支撑日本内需转向正增长。,展望2024年，财政扩张、工资增长和商业信心回暖有望支撑日本内需转向\n正增长。,全球经济金融展望报告\n中国银行研究院 18 2024年\n图12：日本通胀同比增速变化（%...,detailed,5,问题针对报告内容提出了具体且有意义的问题，寻求关于特定主题（2024年日本内需转向正增长的因...


In [51]:
qa_df['score'].value_counts()

score
5    307
4    110
3     11
2      6
Name: count, dtype: int64

In [52]:
qa_df[qa_df['score'] == 3]

,uuid,question,answer,context,doc,qa_type,score,score_reason
32,b560c1be-3b71-46aa-99f3-1c9b58157a0c,哪些因素支撑了发达经济体的增长?,内需是支撑发达经济体增长的主要动力。,从需求端看，内需是支撑发达经济体增长的主要动力，但对经济的拉动作 用逐渐减弱。,全球经济金融展望报告\n中国银行研究院 4 2024年\n图3：部分经济体工业生产指数变化趋...,detailed,3,问题询问了关于发达经济体增长的支撑因素，这是一个好的问题。然而，回答虽然简短但只提到了一个方...
54,b9279752-f66f-4377-b6a8-8135ae78fa73,哪份报告显示了部分发达国家非金融企业部门的债务率？,报告来自IIF和中国银行研究院。,全球经济金融展望报告\n中国银行研究院 6 2024年\n图5：部分发达国家非金融企业部门债...,全球经济金融展望报告\n中国银行研究院 6 2024年\n2023年全球跨境直接投资将继续下...,detailed,3,问题询问了特定的信息来源，回答虽然提供了信息但是并没有明确指出具体的报告名称或位置，因此得分中等。
126,b1c7fe51-98b2-471e-a7ce-6254cc5ef3e2,2010年至2023年间，欧盟对中国哪一种产品发起了反补贴调查?,铜版纸、数据卡、彩涂板、自行车、光伏、太阳能玻璃、玻璃纤维长丝、聚酯短纤维、冷轧不锈钢、热轧...,涉案产品 发起时间...电动汽车 2023/10,全球经济金融展望报告\n中国银行研究院 15 2024年\n和1.2%，这是两个多月的时间里...,detailed,3,问题明确要求了一个具体的产品种类，而回答提供了一连串的产品名称，没有明确指出是哪一种产品在2...
196,684c8686-1a2f-4c4f-a453-e05ef233be43,2023年内风险因素和金融稳定性状态发生了怎样的变化?,2023年内风险因素和金融稳定性状态发生了一系列结构性转换。,年内风险因素和金融稳定性状态发生了一系列结构性转换。,全球经济金融展望报告\n中国银行研究院 25 2024年\n图15：美联储资产规模变化（亿美...,detailed,3,问题询问了具体的事实信息，但回答虽然试图描述一种变化，却缺乏具体细节支持，显得过于笼统。
246,e2d45751-ccaa-4157-a1fd-265e6edcd14e,2024年哪些新兴经济体货币可能面临金融市场压力?,汇率市场动荡且存在政府债务困境的部分新兴经济体,尤其值得关注的是，部分新兴经济体汇率市场动荡，叠加政府债务困境，将引发金融市场压力进一步攀升。,全球经济金融展望报告\n中国银行研究院 31 2024年\n2.新兴经济体货币普遍迎来小幅升...,detailed,3,问题针对未来预测且具体，但回答过于笼统，没有明确指出具体的新兴经济体或货币。
260,9ac73e16-94a7-4d38-9f1f-40e6062ae43f,2023年以来哪些经济体的债券发行规模出现了负增长或低速增长状态?,欧洲、英国等主要经济体,与美国类似，欧洲、英国等主要经济体债券发行规模均呈现负增长或低速增长状态。,全球经济金融展望报告\n中国银行研究院 34 2024年\n（五）债券市场：高利率环境或抑制...,detailed,3,问题针对具体的时间点和具体的经济现象，是有意义的问题。但是回答不够具体详细，没有提供确切的数...
277,d59b0533-714c-47ac-8269-7d56401d8ed7,2024年新能源发展的方向是什么？,2024年新能源发展的重点将是弥补国际原油市场的短缺格局。,国际原油市场短缺格局或延续，新能源发展成为重点。,全球经济金融展望报告\n中国银行研究院 36 2024年\n（六）大宗商品市场：全球大宗商品...,detailed,3,问题针对未来特定时间点的趋势预测，而回答则更多基于当前或过去的逻辑推断，两者之间存在一定的不...
293,3d08b817-1b5e-4d63-ae12-d33e15e2ca93,发布全球经济金融展望报告的机构是哪个?,中国银行研究院,全球经济金融展望报告 中国银行研究院 38 2024年,全球经济金融展望报告\n中国银行研究院 38 2024年\n图24：天然气期货结算价格走势\...,detailed,3,问题本身清晰明确，但是回答内容与问题不符，全球经济金融展望报告通常由国际货币基金组织（IMF...
355,5c2a9e5a-e97e-48d2-a630-d569aa542e44,未来30年房贷利率有何预测?,预计30年房贷利率将在2025年一季度下降到6.4%。,据测算，30年房贷利率将在2025年一季度下降到6.4%。,全球经济金融展望报告\n中国银行研究院 46 2024年\n高级信贷官员的考察发现，银行对所...,detailed,3,问题询问了关于未来房贷利率的预测，这是一个有效的问题。然而，回答只给出了一个具体的预测点而没...
361,3a243458-9410-4608-bfa7-4ad4061de90f,美联储何时可能会开始降息?,2024年,美联储可能在2024年开始降息，房贷利率随之下降，对住房市场需求可能起到一定支撑作用。,全球经济金融展望报告\n中国银行研究院 47 2024年\n第七，美联储加息以来住房贷款质量...,detailed,3,问题询问了一个预测性的事实，回答简洁但缺乏解释依据。


In [53]:
qa_df[qa_df['score'] == 2]

,uuid,question,answer,context,doc,qa_type,score,score_reason
139,163f59fb-2161-40ec-b202-4a473d664240,什么因素导致日本进口价格压力再度上升？,进入三季度后日本面临的进口价格压力再度上升。,进入三季度日本面临的进口价格压力再度上升。,全球经济金融展望报告\n中国银行研究院 17 2024年\n图11：日本GDP环比增长折年率...,detailed,2,问题询问了导致日本进口价格压力上升的因素，但是回答并没有提供这些因素，而是重复了问题中的部分陈述。
151,4c576130-668e-4799-84ec-eff30e5896f5,2023年三季度菲律宾和印度尼西亚的居民消费支出同比增长了多少?,5.1%,新兴经济体增长的主要因素，三季度菲律宾和印度尼西亚居民消费支出同比均增长约5.1%，拉动GD...,全球经济金融展望报告\n中国银行研究院 19 2024年\n新兴经济体增长的主要因素，三季度...,detailed,2,问题询问了具体的数据信息，但是回答没有明确指出哪个国家的消费支出增长了多少，也没有说明这个数...
153,4c576130-668e-4799-84ec-eff30e5896f5,2023年前两个季度巴西和墨西哥的GDP增速如何?,超过3%,拉美新兴经济体表现严重分化，2023年前两个季度巴西和墨西哥GDP增速均超过3%，阿根廷则面...,全球经济金融展望报告\n中国银行研究院 19 2024年\n新兴经济体增长的主要因素，三季度...,detailed,2,问题询问了具体的信息，但是回答太过模糊，没有明确指出哪个国家的具体增速，也没有区分第一季度和...
166,e3f3cbf1-9965-4d35-ac25-ce38281e7e51,‘南南合作’的目标是什么？,'南南合作'的目标是在中国-中亚峰会、金砖国家领导人会晤、“一带一路”国际合作高峰论坛等机制...,在上述机制推动下，未来“南南合作”有望不断增强，共同寻求发展新机遇。,全球经济金融展望报告\n中国银行研究院 20 2024年\n图13：部分亚洲新兴经济体GDP...,detailed,2,回答没有直接说明‘南南合作’的目标，而是提到了一些具体的会议和机制，没有准确回应问题。
268,2fd6cfa9-b7cb-4cb5-8f30-0a44565faa51,2024年哪些债券的收益率预计会维持高位?,相关于美联储、欧洲央行、英格兰银行所持有债券的收益率,2024年，美联储、欧洲央行、英格兰银行将继续推进资产负债表缩减计划，推高相关债券的收益率。,全球经济金融展望报告\n中国银行研究院 35 2024年\n体国债收益率出现较大幅度下跌。当...,detailed,2,问题询问了具体的信息，但是回答并没有直接回答哪些债券的收益率会维持高位，而是提供了一个模糊的...
292,3d08b817-1b5e-4d63-ae12-d33e15e2ca93,2024年的黄金价格是否会因哪些因素得到支撑?,黄金避险功能,黄金避险功能仍将发挥作用，黄金价格底部受到一定支撑。,全球经济金融展望报告\n中国银行研究院 38 2024年\n图24：天然气期货结算价格走势\...,detailed,2,问题询问了关于2024年黄金价格支撑因素的具体内容，但回答只提供了一个非常笼统的概念‘黄金避...


In [54]:
hq_qa_df = qa_df[qa_df['score'] >= 4]

In [55]:
hq_qa_df.shape

(417, 8)

In [56]:
test_q = hq_qa_df.sample(100, replace=False)['question'].values.tolist()

In [57]:
hq_qa_df['dataset'] = 'train'
hq_qa_df.loc[hq_qa_df['question'].isin(test_q), 'dataset'] = 'test'

/tmp/ipykernel_2589713/2938610661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hq_qa_df['dataset'] = 'train'


In [58]:
hq_qa_df.sample(10)

,uuid,question,answer,context,doc,qa_type,score,score_reason,dataset
259,9ac73e16-94a7-4d38-9f1f-40e6062ae43f,2023年美国10年期国债收益率较2022年底提高了多少个基点?,65个基点,截至11月15日，美国、德国、法国、英国、日本的10年期国债收益率分别为4.53%、2.61...,全球经济金融展望报告\n中国银行研究院 34 2024年\n（五）债券市场：高利率环境或抑制...,detailed,5,问题明确具体，针对可量化的事实进行询问。回答直接针对问题给出了具体的数值，没有包含无关信息。,train
187,8d07da80-dc29-4484-979d-02808b092538,2024年美联储计划如何调整其资产负债结构?,2024年美联储计划维持每月950亿美元的缩表节奏，其中包括600亿美元的国债和350亿美元...,美联储将不断调整资产负债结构，维持每月950亿美元（包括600亿美元国债和350亿抵押贷款支...,全球经济金融展望报告\n中国银行研究院 24 2024年\n表2：2023年主要央行加息情况...,detailed,5,问题针对具体的时间点和事件进行询问，回答也直接针对问题给出了明确的信息，没有提及无关内容或提...,train
305,96aec7da-2ee3-4e25-9955-15ecbb8d26ad,2022年沙特的GDP规模是多少？,2022年沙特的GDP规模为1.1万亿美元。,2022年，沙特GDP规模达1.1万亿美元。,全球经济金融展望报告\n中国银行研究院 40 2024年\n注重绿色经济发展，为中海经贸合作...,detailed,5,问题直接明确地询问了一个具体的事实信息，回答也准确地提供了该事实，没有多余的不相关信息。,test
323,c10f0287-041f-4b5d-867b-7d7ed1320c09,沙特的特许经营法案是在哪一年制定的？,2021年。,沙特2021年制定了特许经营法案，,全球经济金融展望报告\n中国银行研究院 42 2024年\n供了空间，进一步坚定了海湾六国经...,detailed,5,问题明确具体，回答精准直接。,test
104,1404f7d2-0948-491a-a6e4-1858f97592de,2023年10月欧元区的通胀率降至多少百分比?,2.9%,通胀率持续下降，但对于居民生活仍带来压力。货币紧缩政策、能源价格回落等因素带动欧元区通胀压力...,全球经济金融展望报告\n中国银行研究院 13 2024年\n图9：欧元区GDP环比增速、综合...,detailed,5,问题明确询问了一个具体的数值信息，回答直接给出了相应的具体数值，没有多余的无关内容。,test
239,e2d45751-ccaa-4157-a1fd-265e6edcd14e,2024年下半年哪些亚洲货币预计会有所回升?,印度卢比、泰铢和韩元,预计2024年下半年，随着美元指数逐渐回落，部分亚洲货币，如印度卢比、泰铢和韩元汇率将有所回升。,全球经济金融展望报告\n中国银行研究院 31 2024年\n2.新兴经济体货币普遍迎来小幅升...,detailed,5,问题明确且具体，针对未来预测提出，回答直接相关且提供了具体的货币名称。,test
31,193f8818-e4e9-4cb4-8649-a865b368e90b,2023年全球债券市场的状况如何，特别是美国债券市场的情况?,2023年全球债券市场在整体货币政策趋紧的大环境下，出现了融资规模下降的现象。在美国市场方面...,,图19：美国联邦基金目标利率与全球MSCI指数\n资料来源：Wind，中国银行研究院\n表3...,large_context,5,问题针对具体的时间点和市场情况提出，具有针对性和实际意义。回答内容直接相关，提供了详细的市场...,test
21,d76771e7-80b8-4efe-bec5-ec850df4769d,2023年全球主要发达经济体的货币政策有何变化，以及新兴经济体采取了何种行动?,2023年，为继续遏制高通胀，主要发达经济体如美国、欧元区及英国的货币政策延续收紧态势，但总...,,表1：2024年全球主要经济体关键指标预测（%）\n地区年\n国家GDP增长率 CPI涨幅 ...,large_context,5,问题针对具体的时间点询问关于全球经济体的货币政策变化，这是一个明确且深入的问题。回答详细地说...,train
250,059c0474-67f3-440e-bc59-fa61172d228c,2023年11月15日前哪些行业在MSCI指数中的涨幅最大?,2023年11月15日之前，信息科技行业的涨幅达到了41.1%，超过了电信业务和非日常消费品...,截至2023年11月15日，信息科技MSCI指数涨幅高达41.1%，远高于电信业务（31.5...,全球经济金融展望报告\n中国银行研究院 33 2024年\n展望2024年，全球股票市场走势...,detailed,5,问题明确询问了特定时间点前涨幅最大的行业，回答直接针对问题提供了具体的信息科技行业的涨幅数据...,train
76,e9ef727f-6276-4854-b77c-546106a48689,2023年美国政府采取了哪些政策来支持经济增长？,2023年美国政府通过实施《通胀削减法案》和《芯片与科学法案》来加大对清洁能源和半导体产业的...,2023年美国政府推进实施《通胀削减法案》《芯片与科学法案》，加\n大对清洁能源、半导体等产...,全球经济金融展望报告\n中国银行研究院 9 2024年\n综合上述因素，“复苏+分化”将成为...,detailed,5,问题明确询问了2023年美国政府为了支持经济增长所采取的具体政策，回答直接针对这个问题提供了...,train


In [59]:
hq_qa_df.to_excel(os.path.join(output_dir, f'question_answer.xlsx'), index=False)